# Técnicas de Score de modelos en Python

Las técnicas de Score (o puntuación) de modelos de ML evalúan su precisión. Habitualmente se basan en realizar algún tipo de comparación entre la salida obtenida y la salida deseada.

Hay muchas técnicas de Score en ML, pero una familia de técnicas muy populares son las basadas en las Confusion matrices. Estas matrices cruzan las predicciones posibles, con las salidas deseadas, estableciendo una clasificación de la salida del modelo como:
- TN: True Negative, el modelo predice correctamente un resultado negativo.
- FN: False Negative, el modelo predice incorrectamente un resultado negativo.
- TP: True Positive, el modelo predice correctamente un resultado positivo.
- FP: False Positive, el modelo predice incorrectamente un resultado positivo.

Las técnicas más comunes populares de este tipo son: precision, recall, f1-score y accuracy.

En Pytorch no se implementa nativamente ninguna técnica de Score. Pero, en el ecosistema de Pytorch, hay un librería estándar de-facto que se puede usar para ello: *torchmetrics*

Este librería tiene un API que implementa varias métricas con una interfaz común que proporciona los métodos: *update()*, *compute()* y *reset()*. Veamos un ejemplo completo de su uso:

In [18]:
!pip install torchmetrics

Defaulting to user installation because normal site-packages is not writeable


In [9]:
import torch
import torch.nn as nn
from torchmetrics import F1Score
from torch.utils.data import DataLoader, TensorDataset

# Definimos un modelo muy simple para clasificación binaria
class BinaryClassificationModel(nn.Module):
    def __init__(self):
        super(BinaryClassificationModel, self).__init__()
        self.linear = nn.Linear(10, 1)
    
    def forward(self, x):
        output = torch.sigmoid(self.linear(x))
        return output

# Inicializamos el modelo, la métrica y el optimizador
model = BinaryClassificationModel()
metric = F1Score(task='binary',num_classes=1, threshold=0.5)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Creamos un dataset de ejemplo aleatorio
inputs = torch.randn(100, 10)
targets = torch.randint(0, 2, (100,)).float()  # Binary targets
dataset = TensorDataset(inputs, targets)
dataloader = DataLoader(dataset, batch_size=10)

# Bucle de entrenamiento
for epoch in range(5):  # 5 épocas
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = nn.BCELoss()(outputs.view(-1), labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # Actualizamos la métrica
        metric(outputs.view(-1), labels)
        
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(dataloader)}")
    print(f"Epoch {epoch+1}, F1-Score: {metric.compute()}")

# Reinicializamos la métrica para la siguiente época
metric.reset()


Epoch 1, Loss: 0.7377187490463257
Epoch 1, F1-Score: 0.6280992031097412
Epoch 2, Loss: 0.7340951383113861
Epoch 2, F1-Score: 0.6255143880844116
Epoch 3, Loss: 0.7308855772018432
Epoch 3, F1-Score: 0.6246575117111206
Epoch 4, Loss: 0.7277696311473847
Epoch 4, F1-Score: 0.6213991641998291
Epoch 5, Loss: 0.7247280299663543
Epoch 5, F1-Score: 0.6181818246841431


También podemos usar una estructura de tipo *MetricCollection* para recopilar dos métricas a la vez. Veamos una adaptación del ejemplo anterior

In [19]:
from torchmetrics import HammingDistance, F1Score, MetricCollection


# Inicializamos el modelo, la métrica y el optimizador
model = BinaryClassificationModel()
metrics = MetricCollection([HammingDistance(task='binary'), F1Score(task='binary',num_classes=1, threshold=0.5)])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Creamos un dataset de ejemplo aleatorio
inputs = torch.randn(100, 10)
targets = torch.randint(0, 2, (100,)).long()  # Binary targets
dataset = TensorDataset(inputs, targets)
dataloader = DataLoader(dataset, batch_size=10)

# Bucle de entrenamiento
for epoch in range(5):  # 5 épocas
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = nn.BCELoss()(outputs.view(-1), labels.float())
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # Actualizamos la métrica
        pred_labels = torch.round(outputs.view(-1))
        metrics(pred_labels, labels)
        
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(dataloader)}")
    print(f"Epoch {epoch+1}, Metrics: {metrics.compute()}")
    # Reinicializamos la métrica para la siguiente época
    metrics.reset()


Epoch 1, Loss: 0.7472673714160919
Epoch 1, Metrics: {'BinaryHammingDistance': tensor(0.5900), 'BinaryF1Score': tensor(0.5203)}
Epoch 2, Loss: 0.7440880835056305
Epoch 2, Metrics: {'BinaryHammingDistance': tensor(0.5900), 'BinaryF1Score': tensor(0.5203)}
Epoch 3, Loss: 0.7412157535552979
Epoch 3, Metrics: {'BinaryHammingDistance': tensor(0.5900), 'BinaryF1Score': tensor(0.5124)}
Epoch 4, Loss: 0.7384244978427887
Epoch 4, Metrics: {'BinaryHammingDistance': tensor(0.5800), 'BinaryF1Score': tensor(0.5167)}
Epoch 5, Loss: 0.7357148230075836
Epoch 5, Metrics: {'BinaryHammingDistance': tensor(0.5700), 'BinaryF1Score': tensor(0.5210)}
